In [ ]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#model selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# specifically for cnn
from keras.layers import Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import random as rn

# specifically for getting numpy arrays of pixel values of images.
import cv2                   
from tqdm import tqdm              
from random import shuffle  

# Preparing the Data

In [ ]:
import os
print(os.listdir("/kaggle/input/flowers-recognition/flowers"))

In [ ]:
X=[]
Z=[]
IMG_SIZE=150
FLOWER_DAISY_DIR='/kaggle/input/flowers-recognition/flowers/daisy'
FLOWER_SUNFLOWER_DIR='/kaggle/input/flowers-recognition/flowers/sunflower'
FLOWER_TULIP_DIR='/kaggle/input/flowers-recognition/flowers/tulip'
FLOWER_DANDI_DIR='/kaggle/input/flowers-recognition/flowers/dandelion'
FLOWER_ROSE_DIR='/kaggle/input/flowers-recognition/flowers/rose'

In [ ]:
def make_train_data(flower_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label = flower_type
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        X.append(np.array(img))
        Z.append(str(label))

In [ ]:
make_train_data('Daisy',FLOWER_DAISY_DIR)
print(len(X))

make_train_data('Sunflower',FLOWER_SUNFLOWER_DIR)
print(len(X))

make_train_data('Tulip',FLOWER_TULIP_DIR)
print(len(X))

make_train_data('Tulip',FLOWER_TULIP_DIR)
print(len(X))

make_train_data('Rose',FLOWER_ROSE_DIR)
print(len(X))

# Data Visualization

In [ ]:
from tensorflow.keras.preprocessing import image
img = image.load_img('/kaggle/input/flowers-recognition/flowers/daisy/100080576_f52e8ee070_n.jpg')
plt.imshow(img)

In [ ]:
fig,ax=plt.subplots(5,2)
fig.set_size_inches(15,15)
for i in range(5):
    for j in range (2):
        l=rn.randint(0,len(Z))
        ax[i,j].imshow(X[l])
        ax[i,j].set_title('Flower: '+Z[l])
        
plt.tight_layout()

# Label Encoding

In [ ]:
label=LabelEncoder()
Y=label.fit_transform(Z)
Y

In [ ]:
Y=to_categorical(Y,5)
X=np.array(X)
X=X/255
Y

# Spliting into Train set and Test set

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)

In [ ]:
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

# Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

# Building the CNN Model

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
 

model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(5, activation = "softmax"))

In [ ]:
batch_size=128
epochs=50

from keras.callbacks import ReduceLROnPlateau
red_lr= ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)

# Compiling the model and Summery

In [ ]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

# Fiting the train data on Model

In [ ]:
History = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)

# Evaluating the Model Performance

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()


In [ ]:
(ls,acc)=model.evaluate(x=x_test,y=y_test)

In [ ]:
(tls, tacc) = model.evaluate(x=x_train, y=y_train)

In [ ]:
print('TRAIN MODEL ACCURACY = {}%'.format(tacc*100))
print('TEST MODEL ACCURACY = {}%'.format(acc*100))

# Model 02

In [ ]:
model2 = Sequential()
model2.add(Conv2D(32,(3,3), padding='same', input_shape=(150,150,3),activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2))) 
model2.add(Conv2D(64,(3,3), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2))) 
model2.add(Conv2D(32,(3,3), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2))) 
model2.add(Flatten())
model2.add(Dense(64,activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(5, activation='softmax'))

model.summary()

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
History = model2.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = 10, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)

In [ ]:
(tls02, tacc02) = model2.evaluate(x=x_train, y=y_train)
(ls02,acc02)=model2.evaluate(x=x_test,y=y_test)

In [ ]:
print('TRAIN MODEL ACCURACY = {}%'.format(tacc02*100))
print('TEST MODEL ACCURACY = {}%'.format(acc02*100))